In [1]:
import coffea
from git import Repo
import awkward as ak
import dask_awkward as dak
import vector
mod = "Prayag Yadav"
local_repo = Repo(path='coffea')
local_branch = local_repo.active_branch.name
test_file = '../../../coffea-fcc-analyses/data/p8_ee_ZH_ecm240/events_082532938.root'
from coffea.nanoevents import NanoEventsFactory, FCCSchema, FCC
print("_______________________________________")
print("\tCurrent Configuration")
print("---------------------------------------")
print("Coffea Version: ", coffea.__version__)
print("Branch: \t", local_branch)
print("Modified by: \t", mod)
print("_______________________________________")


/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/methods/fcc.py:6: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import base, vector


TypeError: Cannot create a consistent method resolution
order (MRO) for bases VectorAwkward, MomentumProtocolSpatial

In [2]:
events = NanoEventsFactory.from_root(
    test_file+":events",
    entry_stop=100,
    schemaclass= FCC.get_schema(zip_missing=False),
    delayed = False
).events()

Jet
ParticleIDs
ReconstructedParticles
EFlowNeutralHadron
MissingET
Particle
MCRecoAssociations
EFlowPhoton
EFlowTrack


In [3]:
events.fields

['AllMuonidx0_collectionID',
 'AllMuonidx0_index',
 'EFlowNeutralHadron',
 'EFlowNeutralHadronidx0_collectionID',
 'EFlowNeutralHadronidx0_index',
 'EFlowNeutralHadronidx1_collectionID',
 'EFlowNeutralHadronidx1_index',
 'EFlowNeutralHadronidx2_collectionID',
 'EFlowNeutralHadronidx2_index',
 'EFlowPhoton',
 'EFlowPhotonidx0_collectionID',
 'EFlowPhotonidx0_index',
 'EFlowPhotonidx1_collectionID',
 'EFlowPhotonidx1_index',
 'EFlowPhotonidx2_collectionID',
 'EFlowPhotonidx2_index',
 'EFlowTrack',
 'EFlowTrackidx0_collectionID',
 'EFlowTrackidx0_index',
 'EFlowTrackidx1_collectionID',
 'EFlowTrackidx1_index',
 'Electronidx0_collectionID',
 'Electronidx0_index',
 'Jet',
 'Jetidx0_collectionID',
 'Jetidx0_index',
 'Jetidx1_collectionID',
 'Jetidx1_index',
 'Jetidx2_collectionID',
 'Jetidx2_index',
 'Jetidx3_collectionID',
 'Jetidx3_index',
 'Jetidx4_collectionID',
 'Jetidx4_index',
 'Jetidx5_collectionID',
 'Jetidx5_index',
 'MCRecoAssociations',
 'MCRecoAssociationsidx0_collectionID',
 'M

In [4]:
events.ReconstructedParticlesidx2_index

<Array [[], [], [], [], ..., [], [], [], []] type='100 * var * int32[parame...'>

In [5]:
Muons = events.ReconstructedParticles.match_collection(events.Muonidx0_index)

In [16]:
Muons[-2].phi

<Array [-1.71, -2.58] type='2 * float32'>